# Memoirr: Preprocessor + Chunker Pipeline Smoke Test

This notebook runs a minimal Haystack pipeline using the SRT preprocessor and the semantic chunker.

Requirements:
- Ensure a local sentence-transformers model is available under `models/<EMBEDDING_MODEL_NAME>/` (with `model.safetensors` and tokenizer/config files).
- `.env` should set `EMBEDDING_MODEL_NAME` (default included in repo). Optionally set `EMBEDDING_DEVICE` (e.g., `cuda:0`).

Notes:
- The preprocessor emits cleaned JSONL lines, one per caption.
- The chunker uses Chonkie SemanticChunker with the self-hosted embeddings to create time-aware chunks.


In [1]:

import pathlib
import textwrap
from src.core.config import get_settings

settings = get_settings()
print('EMBEDDING_MODEL_NAME =', settings.embedding_model_name)
print('EMBEDDING_DEVICE     =', settings.device)

# Quick existence check to help the user
model_path = pathlib.Path('models') / settings.embedding_model_name
if not model_path.exists():
    # Fallback: search by terminal folder name (case-insensitive), similar to runtime resolver
    target = settings.embedding_model_name.split('/')[-1].lower()
    candidates = [p for p in pathlib.Path('models').rglob('*') if p.is_dir() and p.name.lower() == target]
    if candidates:
        print('Found candidate model dir at:', candidates[0])
    else:
        print('WARNING: Expected model folder not found under models/. The chunker cell may fail.')


EMBEDDING_MODEL_NAME = qwen3-embedding-0.6B
EMBEDDING_DEVICE     = 
Found candidate model dir at: models/chunker/qwen3-embedding-0.6B


In [2]:
# Sample SRT content (very small)
sample_srt = textwrap.dedent('''
1
00:00:01,000 --> 00:00:02,000
- Hello there!

2
00:00:02,100 --> 00:00:03,000
How are you doing?

3
00:00:03,100 --> 00:00:04,000
I'm fine. Thanks!
''')
print(sample_srt)



1
00:00:01,000 --> 00:00:02,000
- Hello there!

2
00:00:02,100 --> 00:00:03,000
How are you doing?

3
00:00:03,100 --> 00:00:04,000
I'm fine. Thanks!



In [3]:
# Run the complete end-to-end pipeline: SRT → Preprocess → Chunk → Embed → Qdrant
from src.pipelines.srt_to_qdrant import build_srt_to_qdrant_pipeline

print('Building the complete SRT-to-Qdrant pipeline...')
pipe = build_srt_to_qdrant_pipeline()

print('Pipeline components:')
for component_name in pipe.graph.nodes:
    print(f'  - {component_name}')

print('Pipeline connections:')
for edge in pipe.graph.edges:
    print(f'  - {edge[0]} → {edge[1]}')

print('Running pipeline on sample SRT...')
result = pipe.run({'pre': {'srt_text': sample_srt}})

print('Pipeline Results:')
print('=================')

# Show preprocessing stats
pre_stats = result['pre']['stats']
print(f'Preprocessor: {pre_stats}')

# Show chunking stats
chunk_stats = result['chunk']['stats']
print(f'Chunker: {chunk_stats}')

# Show write stats
write_stats = result['write']['stats']
print(f'Writer: {write_stats}')

print('✅ SUCCESS! Check Qdrant UI at http://localhost:6300/dashboard to see the embedded chunks!')
print('Collection name: memoirr')

2025-09-20T18:59:11.007811Z [info     ] Pipelines module initialized   available_pipelines=['srt_to_qdrant'] environment=development module=pipelines service=memoirr


2025-09-20T18:59:11.227922Z [info     ] Building SRT-to-Qdrant pipeline component=pipeline_builder pipeline_type=srt_to_qdrant


Building the complete SRT-to-Qdrant pipeline...
{'pipeline_type': 'srt_to_qdrant', 'component': 'pipeline_builder', 'event': 'Building SRT-to-Qdrant pipeline', 'level': 'info', 'timestamp': '2025-09-20T18:59:11.227922Z'}


2025-09-20T18:59:14.556358Z [info     ] SRTPreprocessor initialized    component=preprocessor dedupe_window_ms=1000 min_len=1


{'min_len': 1, 'dedupe_window_ms': 1000, 'component': 'preprocessor', 'event': 'SRTPreprocessor initialized', 'level': 'info', 'timestamp': '2025-09-20T18:59:14.556358Z'}


...

{'written_count': 1, 'component': 'qdrant_writer', 'event': 'Documents written successfully', 'level': 'info', 'timestamp': '2025-09-20T18:59:17.688174Z'}


2025-09-20T18:59:17.689410Z [info     ] metric_counter                 component=qdrant_writer metric_name=documents_written_total metric_type=counter status=success value=1


{'metric_name': 'documents_written_total', 'metric_type': 'counter', 'value': 1, 'component': 'qdrant_writer', 'status': 'success', 'event': 'metric_counter', 'level': 'info', 'timestamp': '2025-09-20T18:59:17.689410Z'}


2025-09-20T18:59:17.689988Z [info     ] document_writing_completed     documents_skipped=0 documents_written=1 duration_ms=155 operation=document_writing success_rate=1.0 total_documents=1


{'operation': 'document_writing', 'duration_ms': 155, 'total_documents': 1, 'documents_written': 1, 'documents_skipped': 0, 'success_rate': 1.0, 'event': 'document_writing_completed', 'level': 'info', 'timestamp': '2025-09-20T18:59:17.689988Z'}
Pipeline Results:
Preprocessor: {'total_captions': 3, 'kept': 3, 'dropped_empty': 0, 'dropped_non_english': 0, 'deduped': 0}
Chunker: {'input_captions': 3, 'output_chunks': 1, 'avg_tokens_per_chunk': 15.0}
Writer: {'written': 1, 'skipped': 0, 'total': 1}
✅ SUCCESS! Check Qdrant UI at http://localhost:6300/dashboard to see the embedded chunks!
Collection name: memoirr


In [5]:
# Retrieval Pipeline Test
# This script demonstrates how to use the RAG pipeline to retrieve and generate answers from
# indexed content

from src.pipelines.rag_pipeline import RAGPipeline

# Initialize the RAG pipeline
print("Initializing RAG pipeline...")
rag = RAGPipeline()

# Test basic retrieval with a sample query
print("\nTesting basic retrieval...")
query = "What is the main character's name?"

try:
    result = rag.query(query)

    print(f"Query: {query}")
    print(f"Retrieved documents: {result['summary']['documents_retrieved']}")
    print(f"Generated replies: {result['summary']['replies_generated']}")

    if result.get('answer'):
        print(f"Answer: {result['answer']}")

    # Show source documents
    if result.get('sources'):
        print("\nTop source documents:")
        for i, source in enumerate(result['sources'][:3]):  # Show top 3 sources
            print(f"  {i+1}. {source['content'][:100]}... (score: {source['score']:.3f})")

except Exception as e:
    print(f"Error during retrieval: {e}")

print("\n✅ Retrieval pipeline tests completed!")


2025-09-20T19:06:34.959963Z [info     ] Building RAG pipeline          component=rag_pipeline_builder has_generator_config=False has_retriever_config=False


Initializing RAG pipeline...
{'has_retriever_config': False, 'has_generator_config': False, 'component': 'rag_pipeline_builder', 'event': 'Building RAG pipeline', 'level': 'info', 'timestamp': '2025-09-20T19:06:34.959963Z'}


2025-09-20T19:06:34.962737Z [info     ] QdrantRetriever initialized successfully collection_name=memoirr component=qdrant_retriever qdrant_url=http://localhost:6300 return_embedding=False score_threshold=0.0 top_k=10


{'qdrant_url': 'http://localhost:6300', 'collection_name': 'memoirr', 'top_k': 10, 'score_threshold': 0.0, 'return_embedding': False, 'component': 'qdrant_retriever', 'event': 'QdrantRetriever initialized successfully', 'level': 'info', 'timestamp': '2025-09-20T19:06:34.962737Z'}


2025-09-20T19:06:34.965162Z [info     ] GroqGenerator initialized successfully component=groq_generator max_tokens=1024 model=llama3-8b-8192 stream=False system_prompt_template=default_system.j2 temperature=0.7 top_p=1.0


...

{'metric_name': 'pipeline_errors_total', 'metric_type': 'counter', 'value': 1, 'component': 'rag_orchestrator', 'error_type': 'PipelineRuntimeError', 'event': 'metric_counter', 'level': 'info', 'timestamp': '2025-09-20T19:06:35.047696Z'}


2025-09-20T19:06:35.048110Z [error    ] rag_orchestration_failed       duration_ms=73 error_message="RAG query failed: The following component failed to run:\nComponent name: 'retriever'\nComponent type: 'QdrantRetriever'\nError: Retrieval failed: 'QdrantDocumentStore' object has no attribute 'query_by_embedding'" error_type=RuntimeError operation=rag_orchestration query_length=34


{'operation': 'rag_orchestration', 'duration_ms': 73, 'error_type': 'RuntimeError', 'error_message': "RAG query failed: The following component failed to run:\nComponent name: 'retriever'\nComponent type: 'QdrantRetriever'\nError: Retrieval failed: 'QdrantDocumentStore' object has no attribute 'query_by_embedding'", 'query_length': 34, 'event': 'rag_orchestration_failed', 'level': 'error', 'timestamp': '2025-09-20T19:06:35.048110Z'}


2025-09-20T19:06:35.048601Z [error    ] rag_query_failed               duration_ms=77 error_message="RAG query failed: The following component failed to run:\nComponent name: 'retriever'\nComponent type: 'QdrantRetriever'\nError: Retrieval failed: 'QdrantDocumentStore' object has no attribute 'query_by_embedding'" error_type=RuntimeError operation=rag_query query_length=34


{'operation': 'rag_query', 'duration_ms': 77, 'error_type': 'RuntimeError', 'error_message': "RAG query failed: The following component failed to run:\nComponent name: 'retriever'\nComponent type: 'QdrantRetriever'\nError: Retrieval failed: 'QdrantDocumentStore' object has no attribute 'query_by_embedding'", 'query_length': 34, 'event': 'rag_query_failed', 'level': 'error', 'timestamp': '2025-09-20T19:06:35.048601Z'}
Error during retrieval: RAG query failed: The following component failed to run:
Component name: 'retriever'
Component type: 'QdrantRetriever'
Error: Retrieval failed: 'QdrantDocumentStore' object has no attribute 'query_by_embedding'

✅ Retrieval pipeline tests completed!
